# Solutions for Wrong Format

Here we will explore the following strategies

- Native LLM Support
- Output Parsers


#### Install OpenAI, HuggingFace and LangChain dependencies

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community

# Native LLM Output Response Support

In [18]:
from langchain_ollama import ChatOllama
import json

In [28]:
llm = ChatOllama(
    model="llama3:8b-instruct-fp16",
    temperature=0.01,
    top_p=0.9,
    top_k=40,
    repeat_penalty=1.1,
    base_url="http://172.31.0.1:11434/v1",
    format="json",
    stop=["</s>", "Human:", "AI:"],
    model_kwargs={
        "num_ctx": 2048,
        "num_predict": 256,
        "mirostat_mode": 2,
        "mirostat_tau": 5,
        "mirostat_eta": 0.1
    }
)

In [29]:
prompt = """Provide information about the most recent Champions League final.
Return the answer in valid JSON format with the following fields:
win_team, lose_team, venue, date, score
"""
response = llm.invoke(prompt)

In [30]:
print(json.loads(response.content))


{'win_team': 'Real Madrid', 'lose_team': 'Liverpool', 'venue': 'Wanda Metropolitano', 'date': 'May 26, 2018', 'score': '3-1'}


In [31]:
type(response.content)

str

# Output Parsers

In [32]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


# Define your desired data structure - like a python data class.
class GameDetails(BaseModel):
    win_team: str = Field(description="The winning team in the football game")
    lose_team: str = Field(description="The losing team in the football game")
    venue: str = Field(description="The venue of the football game")
    date: str = Field(description="The date of the football game")
    score: str = Field(description="The score of the football game")

parser = JsonOutputParser(pydantic_object=GameDetails)

In [33]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"win_team": {"title": "Win Team", "description": "The winning team in the football game", "type": "string"}, "lose_team": {"title": "Lose Team", "description": "The losing team in the football game", "type": "string"}, "venue": {"title": "Venue", "description": "The venue of the football game", "type": "string"}, "date": {"title": "Date", "description": "The date of the football game", "type": "string"}, "score": {"title": "Score", "description": "The score of the football game", "type": "string"}}, "required": ["win_team", "lo

In [34]:
from langchain_core.prompts import PromptTemplate

prompt_txt = """
             Who won the Champions league in 2023
             Use the following format when generating the output response

             Output format instructions:
             {format_instructions}
             """

prompt = PromptTemplate.from_template(template=prompt_txt)

In [35]:
llm_chain = (prompt
              |
            llm
              |
            parser)

response = llm_chain.invoke({"format_instructions": parser.get_format_instructions()})

In [36]:
response

{'win_team': 'Real Madrid',
 'lose_team': 'Liverpool',
 'venue': 'Stade de France',
 'date': 'June 10, 2023',
 'score': '1-0'}

In [37]:
type(response)

dict